In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import scipy.ndimage as ndimage
from wrf import(getvar,interplevel, to_np, latlon_coords, interpline, get_cartopy, cartopy_xlim, cartopy_ylim,ALL_TIMES, vertcross, smooth2d, CoordPair, GeoBounds)
from metpy.plots import USCOUNTIES
import metpy.calc as mpcalc
from metpy.units import units
import pandas as pd

############################################
#mpl.rc('text', usetex=True)
mpl.rcParams['text.latex.preamble'] = [r'\boldmath']
############################################

ds = Dataset('wrfout_d03_2018-11-09_01:00:00')
time = getvar(ds, "times",timeidx=ALL_TIMES)

In [ ]:
### Reading in local road shapefiles to be used with the "base map", the roads help to give some spatial scale and awarenes in my opinion
reader = shpreader.Reader('/export/home/mbrewer/wrf_out/shapefiles/tl_2018_06_prisecroads.shp')
roads = list(reader.geometries())                                                                     ## Most major California roadways
roads = cfeature.ShapelyFeature(roads, crs.PlateCarree())

reader = shpreader.Reader('/export/home/mbrewer/wrf_out/shapefiles/tl_2018_06007_roads.shp')
s_roads = list(reader.geometries())                                                                  ### All roads in Butte county.... kinda messy
s_roads = cfeature.ShapelyFeature(s_roads, crs.PlateCarree())

# Function used to create the "base map for all of the plots"
def plot_background(ax):

    ax.coastlines(resolution='10m', linewidth=2, color = 'black', zorder = 4)
    political_boundaries = NaturalEarthFeature(category='cultural',
                                   name='admin_0_boundary_lines_land',
                                   scale='10m', facecolor='none')
    states = NaturalEarthFeature(category='cultural',
                                   name='admin_1_states_provinces_lines',
                                   scale='50m', facecolor='none')

    ax.add_feature(political_boundaries, linestyle='-', edgecolor='black', zorder =4)
    ax.add_feature(states, linestyle='-', edgecolor='black',linewidth=2, zorder =4)
    ax.add_feature(USCOUNTIES.with_scale('500k'), edgecolor='black', linewidth=1, zorder = 1) #### Using Metpy's county shapefiles due to hi-resolution and they also help with spartial awareness
    ax.add_feature(roads, facecolor='none', edgecolor='dimgrey', zorder = 1, linewidth = 1) 
    return ax

##### Funtiction used to calculate the Streamwise component of a wind from a specified angle
def streamwise(Ua,Va, deg = 30):
    
    """ Function used to calculated the streamwise component of the wind base off https://www.eol.ucar.edu/content/wind-direction-quick-reference
     deg
    """
    Ugeo=-1*np.sin(np.deg2rad(deg))
    Vgeo=-1*np.cos(np.deg2rad(deg))
    D=np.arctan2(Vgeo,Ugeo)
    Us=Ua*np.cos(D)+Va*np.sin(D)
    Vs=-Ua*np.sin(D)+Va*np.cos(D)
    return Us,Vs  


def t_ind(Time):
    
    """ Time = Local time
    input time sting in Y-m-d H:M:S format """
    ds_time = pd.to_datetime(time.data).tz_localize('UTC').tz_convert('US/Pacific')
    T = pd.to_datetime(Time, format = '%Y-%m-%d %H:%M:%S').tz_localize('UTC').tz_convert('US/Pacific')
    T_ind = np.where(ds_time == T)
    t = int(T_ind[0])
    ts = str(T)[:-9]
    return t,ts

def VPD(T, RH):
    
    """ T2 can be either kelvin or celcius, the saturation vapor pressure equation requires celcius but there is a fix in the fucntion 
    Saturation Vapor Pressure ased of Bolton 1980:6.112 e^\frac{17.67T}{T + 243.5}
    Vapor Pressure is based of RH = (e/es)*100 ---> e = (RH/100) * es"""
    
    if T.max() > 200:
        T = T -273.15
    svp = 6.112 * np.exp((17.67*T)/(T+243.5))
    vp = (RH/100)*svp
    VPD = svp-vp
    return VPD

In [ ]:
rh2 = getvar(ds, 'rh2', timeidx=ALL_TIMES)
wspd10m, wdir10m = getvar(ds,'uvmet10_wspd_wdir', timeidx = ALL_TIMES)
t2 = ds.variables['T2'][:] ### pulling 2meter temp out seperately since it does not need to be calculated
slp = getvar(ds, "slp",timeidx=ALL_TIMES)
smooth_slp = smooth2d(slp, 3)
vpd = VPD(t2, rh2)

In [ ]:
###########################################################################################
############################    WIND SPEED PLOT FINAL    ##################################
###########################################################################################

wspd10m, wdir10m = getvar(ds,'uvmet10_wspd_wdir', timeidx = ALL_TIMES)
ter = getvar(ds, "ter")


t_list = ['2018-11-09 01:00:00', '2018-11-09 01:10:00','2018-11-09 01:20:00','2018-11-09 01:30:00','2018-11-09 01:40:00','2018-11-09 01:50:00']
for t in t_list:
    t, ts = t_ind(t)
    skip = 10
    cf_var = wspd10m
    lats, lons = latlon_coords(cf_var)
    cart_proj = get_cartopy(cf_var)
    # Create the figure
    fig, ax = plt.subplots(figsize = (30,20),subplot_kw={'projection': cart_proj}, dpi = 300)
    plot_background(ax)


    # Add the color contours
    levels = np.arange(0,30, .5)
    cf = ax.contourf(to_np(lons), to_np(lats), to_np(cf_var[t]), levels = levels, cmap='jet',transform=crs.PlateCarree())
    cb = plt.colorbar(cf, ax=ax, orientation="vertical", pad=.001,label = 25)
    cb.ax.tick_params(labelsize=25)
    cb.set_label('Wind Speed (m$s^{-1}$)', fontsize = 30 , fontweight='bold')
    con_levels = np.arange(0,4000, 250)
    ax.contour(to_np(lons), to_np(lats), to_np(ter),levels = con_levels, colors = 'k',transform=crs.PlateCarree() ,alpha = .5)

    ax.scatter(-121.6219, 39.7596, s =350,  marker = '*', label = 'Paradise, California', transform = crs.PlateCarree(), color = 'k',)
    ax.legend(fontsize = 25, loc =  3)

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(cf_var[0]))
    ax.set_ylim(cartopy_ylim(cf_var[0]))
    #ax.gridlines()

    # Add a title
    #plt.title('WRF %0.1f m'%(ds.DX), loc='left', fontweight='bold', fontsize = 35)
    #plt.title('Color Fill: %s \n Barbs: %s'%(cf_var.description.title(), u.description) , loc='center', fontweight='bold', fontsize = 25)
    #plt.title('%s'%(cf_var.description.title()) , loc='center', fontweight='bold', fontsize = 25)
    plt.title('Valid Time:\n %s PST' % (ts), loc='left', fontweight='bold', fontsize = 25)
    plt.savefig('%s.png'% (cf_var.description + ' ' + str(ds.DX) + ' ' + ts),dpi = 300, bbox_inches = 'tight')
    plt.show()

###########################################################################################
############################    VPD PLOT FINAL    ##################################
###########################################################################################

t2 = ds.variables['T2'][:] ### pulling 2meter temp out seperately since it does not need to be calculated
rh2 = getvar(ds, 'rh2', timeidx=ALL_TIMES)
vpd = VPD(t2, rh2)
u_10, v_10 =getvar(ds, 'uvmet10', timeidx = ALL_TIMES)

t_list = ['2018-11-08 22:00:00', '2018-11-08 22:10:00','2018-11-08 22:20:00','2018-11-08 22:30:00','2018-11-08 22:40:00','2018-11-08 22:50:00']
for t in t_list:
    t, ts = t_ind(t)
    skip = 10
    cf_var = vpd
    u,v = u_10, v_10
    lats, lons = latlon_coords(u)
    cart_proj = get_cartopy(u)
    # Create the figure
    fig, ax = plt.subplots(figsize = (30,20),subplot_kw={'projection': cart_proj}, dpi = 300)
    plot_background(ax)


    # Add the color contours
    levels = np.arange(0,32, .5)
    cf = ax.contourf(to_np(lons), to_np(lats), to_np(cf_var[t]),levels = levels, cmap='viridis',transform=crs.PlateCarree())
    cb = plt.colorbar(cf, ax=ax, orientation="vertical", pad=.001,label = 25)
    cb.ax.tick_params(labelsize=25)
    cb.set_label('Vapor Pressure Deficit (hPa)', fontsize = 30 , fontweight='bold')

    # adding in wind barbs, skip defined above skips that interval of barbs to make the plot more readable 
    plt.barbs(to_np(lons[::skip,::skip]), to_np(lats[::skip,::skip]),
              to_np(u[t,::skip,::skip]), to_np(v[t,::skip,::skip]),
              transform=crs.PlateCarree(), length=8, linewidth = 2)
    ax.scatter(-121.6219, 39.7596, s =350,  marker = '*', label = 'Paradise, California', transform = crs.PlateCarree(), color = 'k',)
    ax.legend(fontsize = 25, loc =  3)

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(u[0]))
    ax.set_ylim(cartopy_ylim(u[0]))
    #ax.gridlines()

    # Add a title
    #plt.title('WRF %0.1f m'%(ds.DX), loc='left', fontweight='bold', fontsize = 35)
    #plt.title('Color Fill: %s \n Barbs: %s'%(cf_var.description.title(), u.description) , loc='center', fontweight='bold', fontsize = 25)
    #plt.title('%s'%(cf_var.description.title()) , loc='center', fontweight='bold', fontsize = 25)
    plt.title('Valid Time:\n %s PST' % (ts), loc='left', fontweight='bold', fontsize = 25)
    plt.savefig('%s.png'% ('VPD' + '_' + str(ds.DX) + '_' + ts),dpi = 300, bbox_inches = 'tight')
    plt.show()

In [ ]:
###########################################################################################
############################    RH PLOT FINAL    ##################################
###########################################################################################


rh2 = getvar(ds, 'rh2', timeidx=ALL_TIMES)
u_10, v_10 =getvar(ds, 'uvmet10', timeidx = ALL_TIMES)

t_list = ['2018-11-08 23:00:00', '2018-11-08 23:10:00','2018-11-08 23:20:00','2018-11-08 23:30:00','2018-11-08 23:40:00','2018-11-08 23:50:00']
for t in t_list:
    t, ts = t_ind(t)
    skip = 10
    cf_var = rh2
    u,v = u_10, v_10
    lats, lons = latlon_coords(cf_var)
    cart_proj = get_cartopy(cf_var)
    # Create the figure
    fig, ax = plt.subplots(figsize = (30,20),subplot_kw={'projection': cart_proj}, dpi = 300)
    plot_background(ax)


    # Add the color contours
    levels = np.arange(0,75, 5)
    cf = ax.pcolormesh(to_np(lons), to_np(lats), to_np(cf_var[t]), cmap='nipy_spectral_r',transform=crs.PlateCarree(), vmin = 0, vmax = 65)
    cb = plt.colorbar(cf, ax=ax, orientation="vertical", pad=.001,label = 25,ticks = np.arange(0,cf_var.max(),5))
    cb.ax.tick_params(labelsize=25)
    cb.set_label('2-m Relative Humidity (%)', fontsize = 30 , fontweight='bold')

    # adding in wind barbs, skip defined above skips that interval of barbs to make the plot more readable 
    plt.barbs(to_np(lons[::skip,::skip]), to_np(lats[::skip,::skip]),
              to_np(u[t,::skip,::skip]), to_np(v[t,::skip,::skip]),
              transform=crs.PlateCarree(), length=8, linewidth = 2)
    ax.scatter(-121.6219, 39.7596, s =350,  marker = '*', label = 'Paradise, California', transform = crs.PlateCarree(), color = 'k',)
    ax.legend(fontsize = 25, loc =  3)

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(cf_var[0]))
    ax.set_ylim(cartopy_ylim(cf_var[0]))
    #ax.gridlines()

    # Add a title
    #plt.title('WRF %0.1f m'%(ds.DX), loc='left', fontweight='bold', fontsize = 35)
    #plt.title('Color Fill: %s \n Barbs: %s'%(cf_var.description.title(), u.description) , loc='center', fontweight='bold', fontsize = 25)
    #plt.title('%s'%(cf_var.description.title()) , loc='center', fontweight='bold', fontsize = 25)
    plt.title('Valid Time:\n %s PST' % (ts), loc='left', fontweight='bold', fontsize = 25)
    plt.savefig('%s.png'% (cf_var.description + ' ' + str(ds.DX) + ' ' + ts),dpi = 300, bbox_inches = 'tight')
    plt.show()

In [ ]:
###########################################################################################
############################     DEW POINT PLOT FINAL    ##################################
###########################################################################################


td2 = getvar(ds, 'td2', timeidx=ALL_TIMES)
u_10, v_10 =getvar(ds, 'uvmet10', timeidx = ALL_TIMES)

t_list = ['2018-11-09 01:00:00', '2018-11-09 01:10:00','2018-11-09 01:20:00','2018-11-09 01:30:00','2018-11-09 01:40:00','2018-11-09 01:50:00']
for t in t_list:
    t, ts = t_ind(t)
    skip = 10
    cf_var = td2
    u,v = u_10, v_10
    lats, lons = latlon_coords(cf_var)
    cart_proj = get_cartopy(cf_var)
    # Create the figure
    fig, ax = plt.subplots(figsize = (30,20),subplot_kw={'projection': cart_proj}, dpi = 300)
    plot_background(ax)


    # Add the color contours
    levels = np.arange(-30,10, 1)
    cf = ax.contourf(to_np(lons), to_np(lats), to_np(cf_var[t]),levels = levels, cmap='jet_r',transform=crs.PlateCarree(), vmax = cf_var.max()-5)
    cb = plt.colorbar(cf, ax=ax, orientation="vertical", pad=.001,label = 25)
    cb.ax.tick_params(labelsize=25)
    cb.set_label('2-m Dew Point Temp ($\degree$C)', fontsize = 30 , fontweight='bold')

    # adding in wind barbs, skip defined above skips that interval of barbs to make the plot more readable 
    plt.barbs(to_np(lons[::skip,::skip]), to_np(lats[::skip,::skip]),
              to_np(u[t,::skip,::skip]), to_np(v[t,::skip,::skip]),
              transform=crs.PlateCarree(), length=8, linewidth = 2)
    ax.scatter(-121.6219, 39.7596, s =350,  marker = '*', label = 'Paradise, California', transform = crs.PlateCarree(), color = 'k',)
    ax.legend(fontsize = 25, loc =  3)

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(cf_var[0]))
    ax.set_ylim(cartopy_ylim(cf_var[0]))
   # ax.gridlines()

    # Add a title
    #plt.title('WRF %0.1f m'%(ds.DX), loc='left', fontweight='bold', fontsize = 35)
    #plt.title('Color Fill: %s \n Barbs: %s'%(cf_var.description.title(), u.description) , loc='center', fontweight='bold', fontsize = 25)
    #plt.title('%s'%(cf_var.description.title()) , loc='center', fontweight='bold', fontsize = 25)
    plt.title('Valid Time:\n %s PST' % (ts), loc='left', fontweight='bold', fontsize = 25)
    plt.savefig('%s.png'% (cf_var.description + ' ' + str(ds.DX) + ' ' + ts),dpi = 300, bbox_inches = 'tight')
    plt.show()

z = getvar(ds, 'z',timeidx=ALL_TIMES)
ua = getvar(ds, 'ua', units ='ms-1',timeidx=ALL_TIMES)
va = getvar(ds, 'va', units ='ms-1',timeidx=ALL_TIMES)
rh = getvar(ds, 'rh',timeidx=ALL_TIMES)
wspd = getvar(ds, 'wspd_wdir', units ='ms-1',timeidx=ALL_TIMES)[0] ## pulling just the wind speed and not the direction
p = getvar(ds, 'p', units = 'hpa', timeidx=ALL_TIMES)
td = getvar(ds, 'td', timeidx=ALL_TIMES)

ht_700 = interplevel(z, p, 700)
u_700 = interplevel(ua, p, 700)
v_700 = interplevel(va, p, 700)
wspd_700 = interplevel(wspd, p, 700)
rh_700 = interplevel(rh, p, 700)
td_700 = interplevel(td, p, 700)

t, ts = t_ind('2018-11-08 16:00:00')
skip = 10
cf_var = rh_700
lats, lons = latlon_coords(cf_var)
cart_proj = get_cartopy(cf_var)
# Create the figure
fig, ax = plt.subplots(figsize = (20,20),subplot_kw={'projection': cart_proj}, dpi = 300)
plot_background(ax)


# Add the color contours
levels = np.arange(cf_var.min(), 10, 1)
cf = ax.contourf(to_np(lons), to_np(lats), to_np(cf_var[t]),levels = levels, cmap='seismic_r',transform=crs.PlateCarree(),)
plt.colorbar(cf, ax=ax, orientation="vertical", pad=.05, shrink = .74)

# Add the 700 hPa wind barbs, only plotting every 110th data point.
plt.barbs(to_np(lons[::skip,::skip]), to_np(lats[::skip,::skip]),
          to_np(u_700[t,::skip,::skip]), to_np(v_700[t,::skip,::skip]),
          transform=crs.PlateCarree(), length=6)
ax.scatter(-121.6219, 39.7596, s =300,  marker = '*', label = 'Paradise, California', transform = crs.PlateCarree(), color = 'tab:red',)

# Set the map bounds
ax.set_xlim(cartopy_xlim(cf_var[0]))
ax.set_ylim(cartopy_ylim(cf_var[0]))

ax.gridlines()

plt.title("700hPa Dewpoint ($\degree$C),  Wind Barbs (kt)")

plt.show()

In [ ]:
ter = getvar(ds, 'ter', units ='m',timeidx=ALL_TIMES)
theta =  getvar(ds, "th",timeidx=ALL_TIMES,)
td =  getvar(ds, "td",timeidx=ALL_TIMES)
z=getvar(ds,'z',timeidx=ALL_TIMES)
z_levels=z[0,:30,0,0] #this limits vertical extent of plot to first 30 levels
U,V=getvar(ds,'uvmet',timeidx = ALL_TIMES) #U and V through the height of the atmosphere

U_s,V_s = streamwise(U,V) ## Calculating streamwise wind @30
t_list = ['2018-11-09 01:00:00', '2018-11-09 01:10:00','2018-11-09 01:20:00','2018-11-09 01:30:00','2018-11-09 01:40:00','2018-11-09 01:50:00']
for t in t_list:
    t, ts = t_ind(t)
    skip = 10
    cf_var = U_s
    con_var  = theta

    # Set the start point and end point for the cross section
    start_point = CoordPair(lat=39.2, lon=-122.1)
    end_point = CoordPair(lat=40.15, lon=-121.3)
    ter_line = interpline(ter[0], wrfin=ds, start_point=start_point,end_point=end_point, latlon=True, meta=False)

    theta_cross = vertcross(con_var[t], z[t],wrfin=ds,  start_point=start_point,
                        end_point=end_point, latlon=True, meta=True, autolevels = 150)
    td_cross = vertcross(cf_var[t], z[t], wrfin=ds, start_point=start_point,
                           end_point=end_point, latlon=True, meta=True,autolevels = 150)
    stream_cross = vertcross(U_s[0], z[0], wrfin=ds, start_point=start_point,
                           end_point=end_point, latlon=True, meta=True,autolevels = 150)
    fig = plt.figure(figsize=(30,20))
    ax = fig.add_subplot(1,1,1)
     #Make the contour plot for wind speed
    clevs = np.arange(-5,30,1)
    w_contours = ax.contourf(to_np(stream_cross),levels = clevs, cmap="jet")
    # Add the color bar
    cb = fig.colorbar(w_contours, ax=ax, pad = 0.01)
    cb.ax.tick_params(labelsize=30)
    cb.set_label('Streamwise winds (m$s^{-1}$)', fontsize = 35 )

    # Make the contour plot for dbz
    lev = np.arange(200, 350, 3)
    theta_contours = ax.contour(to_np(theta_cross), levels = lev, colors = 'k')
    ax.clabel(theta_contours,theta_contours.levels[::2], inline =1, colors='black',fontsize = 25, fmt = '%.1f')

    # Set the x-ticks to use latitude and longitude labels
    # Set the x-ticks to use latitude and longitude labels
    coord_pairs = to_np(theta_cross.coords["xy_loc"])
    x_ticks = np.arange(coord_pairs.shape[0])
    x_labels = [pair.latlon_str(fmt='{:.2f}, {:.2f}') for pair in to_np(coord_pairs)]

    # Set the desired number of x ticks below
    num_ticks = 5
    thin = int((len(x_ticks) / num_ticks) + .5)
    ax.set_xticks(x_ticks[::thin])
    ax.set_xticklabels(x_labels[::thin], rotation=45, fontsize=30)

    # Set the y-ticks to be height
    vert_vals = to_np(theta_cross.coords["vertical"])
    vert_vals = np.round(vert_vals, decimals= 0)
    v_ticks = np.arange(vert_vals.shape[0])
    ax.set_yticks(v_ticks[::1])
    ax.set_yticklabels(vert_vals[::1], fontsize=30)
    plt.ylim(0, 30)
    ###
    ax.set_xlabel("Latitude, Longitude", fontsize=30, fontweight = 'bold')
    ax.set_ylabel("Height (m)", fontsize=30, fontweight = 'bold')


    # Add a title
    #plt.title('WRF %0.1f m'%(ds.DX), loc='left', fontweight='bold', fontsize = 35)
    #plt.title('Color Fill: %s\n Contours: %s'%(cf_var.description.title(),con_var.description.title()), loc='center', fontweight='bold', fontsize = 30)
    plt.title('Valid Time:\n %s PST' % (ts), loc='left', fontweight='bold', fontsize = 35)


    ##### Plot elevation under the x-sec and fill under
    ax1=ax.twinx()
    offset=86 #lift terrain to cover white terrain blocks
    x_length=np.arange(len(ter_line))
    ax1.plot(to_np(ter_line)+offset,color='white',linewidth=2.0) #terrain outline
    ax1.fill_between(x_length,to_np(ter_line)+offset,vert_vals[0],color='dimgrey') #terrain fill
    ax1.set_ylim(0,4000) #lower or raise your terrain here to get it perfect
    plt.xlim(0,len(ter_line)-1)
    plt.yticks([])
    ax1.plot(220, 542, markersize = 50, marker = '.', label = 'Paradise, California', color = 'tab:red',zorder = 5)
    ax1.legend(fontsize = 'xx-large')


    plt.tight_layout()

    plt.savefig('%s.png'% ('xsec_windspeed_theta'+ str(ds.DX) + '_' + ts),dpi = 300, bbox_inches = 'tight')
    plt.show()


In [ ]:
ter = getvar(ds, 'ter', units ='m',timeidx=ALL_TIMES)


# Create the figure
fig = plt.figure(figsize=(30,25))
ax = plt.axes(projection=cart_proj)

# Download and add the states and coastlines
states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
ax.add_feature(states, linewidth=0.5, edgecolor="black")
ax.coastlines('50m', linewidth=0.8)

# Add the wind speed contours
levels = np.arange(-10, 4000, 50)
contours = ax.contourf(to_np(lons), to_np(lats), to_np(ter[0]),levels =levels, cmap='terrain',transform=crs.PlateCarree(),)
cb = fig.colorbar(contours, ax=ax, orientation="vertical", pad=.05, shrink = .74)
cb.set_label('Elevation (m)', size = 'x-large', fontsize = 25 )
cb.ax.tick_params(labelsize=20)
# Set the map bounds
ax.set_xlim(cartopy_xlim(ht_700))
ax.set_ylim(cartopy_ylim(ht_700))

start_point = CoordPair(lat=39.2, lon=-122.1)
end_point = CoordPair(lat=40.15, lon=-121.3)
ax.plot([start_point.lon, end_point.lon],
            [start_point.lat, end_point.lat], color="yellow", marker="o",
            transform=crs.PlateCarree(), zorder=3)

ax.scatter(-121.6219, 39.7596, s =300,  marker = '*', label = 'Paradise, California', transform = crs.PlateCarree(), color = 'tab:red',)

ax.gridlines()

plt.title("Domain #3 Model Terrain", fontsize = 40, fontweight = 'bold')
plt.savefig('xsec1.png', dpi = 300)
plt.show()

In [ ]:
lons[-100,-1]